In [1]:
# Needed to import custom code from other directories
import sys
sys.path.append('../../code')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm.auto import tqdm

from models import ModifiedMLPClassifier
from utils import LRAP, perfection

%matplotlib inline

SEED = 42

NUM_FEATURES = 5000
NUM_CLASSES = 3993

X_train = pd.read_csv("../../data/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("../../data/expanded/train_labels.csv", names=range(NUM_CLASSES))

X_valid = pd.read_csv("../../data/expanded/valid_features.csv", names=range(NUM_FEATURES))
y_valid = pd.read_csv("../../data/expanded/valid_labels.csv", names=range(NUM_CLASSES))

print(f"{X_train.shape[0]} examples in training set, and {X_valid.shape[0]} in validation.")

13959 examples in training set, and 1552 in validation.


## Pre-processing data

In [2]:
# Set True to apply log transformation to the features
# Intended to be used in combination with one of the other techniques below
LOG = True

STANDARDIZE = True     # Set True to standardize, False otherwise
NORMALIZE = False      # Set True to normalize, False otherwise
PCA_COMPONENTS = None  # Set to number of components for PCA (automatically standardized)


# ----- Code below applies the transformations as specified ----- #
func = []
if LOG:
    X_train_transformed = np.log1p(X_train.to_numpy())
    X_valid_transformed = np.log1p(X_valid.to_numpy())
else:
    X_train_transformed = X_train.to_numpy()
    X_valid_transformed = X_valid.to_numpy()

if STANDARDIZE:
    func = [StandardScaler()]
elif NORMALIZE:
    func = [MinMaxScaler()]
elif PCA_COMPONENTS is not None:
    func = [StandardScaler(with_std=False)]
    func.append(PCA(n_components=PCA_COMPONENTS, random_state=SEED))
    func.append(StandardScaler(with_mean=False))
else:
    func = None

if func is not None:
    for f in func:
        f = f.fit(X_train_transformed)
        X_train_transformed = f.transform(X_train_transformed)
        X_valid_transformed = f.transform(X_valid_transformed)

## Model training

While training the multi-layer perceptron, the training loss and validation LRAP are reported in each iteration.

In [3]:
mlp = ModifiedMLPClassifier(
    hidden_layer_sizes=(256,),
    activation="relu",
    max_iter=100,
    alpha=0.2,
    early_stopping=True,
    verbose=True,
    random_state=SEED,
    custom_validation_data=(X_valid_transformed, y_valid)
)
mlp = mlp.fit(X_train_transformed, y_train)

Iteration 1, loss = 302.07358861
Validation score: 0.082953
Iteration 2, loss = 43.54626572
Validation score: 0.155626
Iteration 3, loss = 34.44382504
Validation score: 0.213677
Iteration 4, loss = 30.22854281
Validation score: 0.268971
Iteration 5, loss = 27.38173244
Validation score: 0.312964
Iteration 6, loss = 25.15300299
Validation score: 0.350610
Iteration 7, loss = 23.25748569
Validation score: 0.382617
Iteration 8, loss = 21.65555288
Validation score: 0.406718
Iteration 9, loss = 20.24598316
Validation score: 0.429887
Iteration 10, loss = 18.98345092
Validation score: 0.447184
Iteration 11, loss = 17.84287442
Validation score: 0.464512
Iteration 12, loss = 16.79113709
Validation score: 0.480273
Iteration 13, loss = 15.81512634
Validation score: 0.491235
Iteration 14, loss = 14.91339770
Validation score: 0.501322
Iteration 15, loss = 14.07062573
Validation score: 0.512148
Iteration 16, loss = 13.28946981
Validation score: 0.520505
Iteration 17, loss = 12.55124821
Validation scor

## Evaluation

In [4]:
score_train = mlp.score(X_train_transformed, y_train.to_numpy())
print(f"LRAP on training data: {score_train:.4f}")

LRAP on training data: 0.9718


In [5]:
score_valid = mlp.score(X_valid_transformed, y_valid.to_numpy())
print(f"LRAP on validation data: {score_valid:.4f}")

LRAP on validation data: 0.5757


In [ ]:
print(f"{perfection(y_valid, predictions_valid) * 100:.3f}% of examples perfectly predicted")

In [9]:
sample_idx = 123
sample_pred = mlp.predict(X_valid_transformed[[sample_idx]])[0]
actual = y_valid.loc[sample_idx].to_numpy()

In [10]:
np.nonzero(actual)

(array([342, 418, 637, 682, 906, 992], dtype=int64),)

In [11]:
np.nonzero(sample_pred)

(array([128, 263, 342, 358, 637, 682, 992], dtype=int64),)